## Sommaire

1. Préparation de l'environnement

2. Étude des données Covid19 reportées en Suisse

    2.1 Télécharger
    
    2.2 Comprendre la structure de la donnée
    
    2.3 Préparation de la donnée Covid19 avec Pandas
    
    2.4 Visualisation simple de la donnée
    
    2.5 Corrélations
    
    2.6 Aggrégations des nuages de points

3. Exercices

4. Annexe: méthodes avancées

## 1. Préparation de l'environnement

Nous allons utiliser principalement les fonctions de pandas avec un backend de visualisation matplotlib.

Les paquets Python suivants doivent être installés:
- matplotlib
- numpy
- pandas
- seaborn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
%matplotlib inline
pd.set_option("display.precision",2)

------
## 2. Étude des données Covid19 de Suisse

#### 2.1 Télécharger

Téléchargement de la donnée de Covid19 en Suisse.

* Source: [openZH/covid_19](https://github.com/openZH/covid_19/)
* Format de lecture: CSV
* Méthode de lecture: [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)
     - `index_col`: Indexer la date
     - `usecols`: Selectionne un sous ensemble des colonnes de la table
     - `sort_index`: Arrange chronologiquement
* Renome colonne _abbreviation_canton_and_fl_ en _canton_


In [ ]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/openZH/covid_19/master/COVID19_Fallzahlen_CH_total_v2.csv',
    usecols=['abbreviation_canton_and_fl',
              'date',      
              'ncumul_tested',
              'ncumul_conf',
              'new_hosp',
              'current_hosp',
              'current_icu',
              'current_vent',
              'ncumul_released',
              'ncumul_deceased',
              'current_isolated',
              'current_quarantined'
             ],
    parse_dates=['date'],
    index_col=['date']
).sort_index(ascending=True)
df.rename(columns={"abbreviation_canton_and_fl":"canton"},inplace=True)

* Ajoute une colonne _ncumul_hosp_ indiquant la somme cummulée (`.cumsum()`) des nouvelles hospitalisations _new_hosp_, par canton (`groupby(['canton']`)

In [ ]:
df['ncumul_hosp']=df.groupby('canton').new_hosp.cumsum()

------
#### 2.2 Comprendre la structure de la donnée

Statistiques sommaires de la donnée Covid19 Suisse.

* Des colonnes sont plus complètes que d'autres - lesquelles?

In [ ]:
df.info()

-----
Aperçu de la donnée Covid19

* Échantillon des 5 entrées les plus anciennes et les 5 plus récentes - notez la structure de la table

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

* Dans quels cantons et a quels moments y-a-t-il eu le plus d'hospitalisations dues au Covid19 (top 10)?

In [ ]:
df.nlargest(n=10,columns='current_hosp')[['canton','current_hosp']]

-----
#### 2.3 Préparation de la donnée

Nous voulons visualiser et comparer la série temporelle de chaque mesure sur tous les cantons. Pour ce faire nous effectuons d'abord un _pivot_ de la DataFrame, afin d'arranger les cantons de la colonne `canton` en colonnes séparées.

* Méthode: [pandas.DataFrame.pivot](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_index.html)
    - `columns`: colonne du pivot - chaque nouvelle valeur de cette colonne de la table d'origine résulte en une nouvelle colonne dans la table transformée
    - `values`: liste de colonnes dans la table d'origine contenant les mesures à traiter.
    
Il existe bien sûre d'autres façons de préparer les données pour cette première visualisation - à vous d'explorer. Pour plus de détails, voir les méthodes de [remodelage des DataFrames pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)

In [ ]:
dfp=pd.pivot(df,columns='canton',values=['current_hosp','ncumul_deceased'])

-----
Afficher les informations au sujet de cette nouvelle DataFrame

* Notes:
    - L'index des lignes reste la colonne `date`.
    - L'index des colonnes a deux niveaux _(mesure, canton)_ - par exemple: `(current_hosp, AG)`

In [ ]:
dfp.info()

In [ ]:
dfp.head(5)

-----
#### 2.4 Visualisation de la donnée

Une visualisation nous donne une information plus compacte et plus riche que les aperçus sur des échantillons de données vus précédemments.

Il est possible d'invoquer les méthodes de visualisation directement sur les DataFrames pandas, sans utiliser les fonctions de matplotlib.
Dans ce cas pandas utilise par défaut l'engin de visualisation matplotlib, qu'il est possible de remplacer
à l'aide de l'option [plotting.backend](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html).

* Méthode: [pandas.DataFrame.plot()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html).current_hosp.plot(figsize=(10,10))
    - `current_hosp`: sélectionne toutes les colonnes `current_hosp` (niveau 0 de l'index des colonnes)
    - `figsize=(10,10)`: redimensionne la figure


* Notes:
    - La notation `plot(x='colonne1',y='colonne2',...)` permet de spécifier les colonnes de l'axe _x_ et/ou _y_
    - Par défaut l'index de ligne est utilisé pour _x_ et l'index des colonnes pour _y_ (une courbe par colonne).


In [ ]:
dfp.current_hosp.plot(figsize=(10,10))

-----
Les courbes précédentes présentent deux vagues de l'épidémie Covid19.

Nous nous intéressons à la deuxième vague. Nous lissons la donnée avec une moyenne hébdomadaire afin d'éliminer les fluctuations périodiques liées au facteur humain.

* Méthode
    - [.loc['2020-09-01':]](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#selection-by-label): limite la période à partir du 1 septembre 2020.
        - Une autre façon serait d'utiliser [query('date>"2020-08-31"')](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html#pandas.DataFrame.query).
    - `resample('W').mean()`: mesures hebdomadaires moyennes.


In [ ]:
dfp2vm=dfp.loc['2020-09-01':].resample('W').mean()
dfp2vm.current_hosp.plot(figsize=(10,10))

-----
#### 2.5 Correlations

_Quel est le taux de correlation de l'évolution du covid19 entre les cantons?_


Nous pouvons estimer la présence de correlations en comparant visuellement les mesures d'un canton avec les mesures d'un autre canton prises les mêmes semaines. Nous utilisons des figures de type _nuages de points_ pour indiquer le degré de corrélation entre deux ou plusieurs variables.

In [ ]:
import numpy.polynomial.polynomial as poly
L=poly.polyfit(dfp2vm.current_hosp.ZH,dfp2vm.current_hosp.BE,1)

fig,axe=plt.subplots()
dfp2vm.current_hosp.plot(x='ZH',y='BE',ax=axe,figsize=(10,10),style='b^')

x=np.array([0,350])
y=L[1]*x+L[0]

axe.plot(x,y,'k-')



Pandas permet de créer une matrice de comparaisons paire à paire des ensemble de données

* Méthode: [pandas.plotting.scatter_matrice]()


In [ ]:
pd.plotting.scatter_matrix(dfp2vm.current_hosp[['ZH','BE','VD','VS','NE','GE']],figsize=(10,10))

L'interface DataFrames de pandas offre aussi un ensemble de fonctions qui permettent de calculer les correlations.

* Méthode: [pandas.DataFrame.corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)
    - `corr(method='pearson')`: corrélation du nombre d'hospitalisations hebdomadaire moyen par paire de cantons.

La méthode de _pearson_ est utilisée par défaut - autres choix: _spearman_, _kendall_, ou votre méthode (fonction python)

In [ ]:
correlations=dfp2vm.current_hosp.corr(method='pearson')
correlations

-----
Cette corrélation est visuellement plus parlante avec une _carte thermique_.

Avec matplotlib, un peu de programmation est requise pour nommer les axes et styliser la figure.

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
colonnes=correlations.columns.to_list()
plt.xticks(range(len(colonnes)),colonnes,rotation=90)
plt.yticks(range(len(colonnes)),colonnes)
plt.imshow(correlations,cmap='hot')

Le paquet seaborn offre des racourcis matplotlib pour permettre de créer ce type de figures plus rapidement.

In [ ]:
fig,axe=plt.subplots(figsize=(10,10))
sns.heatmap(correlations,vmin=0.25)

-----
#### 2.6 Aggrégations de nuages de points

L'affichage [pandas.DataFrame.plot.hexbin](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.hexbin.html) permet d'exprimer sous forme de carte thermique la densité des nuages de points en les partitionnant dans des zones d'une grille hexagonales.

Ceci est illustré avec le nombre d'hospitalisations versus le taux d'occupation dans les unités de soins intensifs de chaque canton.

Méthode:

* `reset_index().set_index(['date','canton'])`: retire l'index existant, et index les colonnes _date_ et _canton_, ce qui revient à dire: _appliquer l'opération suivante pour chaque date et chaque canton_.
* `plot(x='current_hosp',y='current_icu')`: pour chaque valeur de l'index, afficher un point correspondant au nombre d'hospitalisations en cours  (axe horizontal) et nombre de patients en ICU.

In [ ]:
df.reset_index().set_index('date','canton').plot(x='current_hosp',y='current_icu',figsize=(10,10),style='b^')

In [ ]:
df.reset_index().set_index('date','canton').plot.hexbin(x='current_hosp',y='current_icu',figsize=(10,10),gridsize=10,vmax=500)

----
## Exercices

#### 4.1 Apprenez vous à décomposer les DataFrames pandas

Devinez pour chacune des commandes suivantes quels ensembles de données sont affichés. Pouvez vous compléter la table? Les réponse aux trois premières questions sont déjà indiquées.
Pour cet exercice vous devez vous assurer d'avoir bien crée la DataFrame _dfp_ comme indiqué au cours de cette séance.

|commande|current_hosp|ncumul_deceased|canton BE|canton ZH|Et autres cantons|Erreur|
|-|-:|-:|-:|-:|-:|-:|
|dfp.plot(legend=False)|X|X|X|X|X||
|dfp['current_hosp'].plot()|X||X|X|X||
|dfp['current_hosp'].ZH.plot()|X|||X|||
|dfp['current_hosp'][['ZH','BE']].plot()||||||
|dfp[['current_hosp']].plot()||||||
|dfp[['current_hosp','ncumul_deceased']].plot()||||||
|dfp[['current_hosp','ncumul_deceased']][['ZH','BE']].plot()||||||
|dfp[[('ncumul_deceased','ZH'),('current_hosp','VD')]].plot()||||||
|dfp.current_hosp.ZH.plot()||||||
|dfp.current_hosp[['ZH','BE']].plot()||||||
|dfp.current_hosp.plot(y='ZH')||||||
|dfp.current_hosp.plot(y=['ZH','BE'])||||||
|dfp.plot(y='current_hosp')||||||
|dfp.plot(y=['current_hosp','ncumul_deceased'],legend=False)||||||
|dfp.plot(y=('current_hosp','AG'))||||||
|dfp.plot(y=[('current_hosp','ZH'),('ncumul_deceased','ZH')])||||||

-----


Utilisez l'espace ci-dessous pour vérifier vos réponses.

Notes:
* la fonction `plt.subplots(2,3)` prépare une grille de $2x3$ et retourne une matrice d'axes. Pour modifier la figure de la ligne $i$, colonne $j$ de la grille il faut passer l'axe[$i,j$] au paramètre `ax` de la fonction `panda.DataFrames.plot`.
* Pour plus d'infos, voir [le tutoriel Pandas d'indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html).


In [ ]:
fig,axes=plt.subplots(2,3,figsize=(20,10))

dfp.plot(ax=axes[0,0],legend=False)
dfp['current_hosp'].plot(ax=axes[0,1])

plt.show()

#### 4.2 Apprendre le principe du pivot

Faites un pivot de la DataFrame _df_, de manière à pouvoir visualiser les mesures _current_icu_ par canton.

-----


-----
## 4. Annexe: méthodes avancées


Pour les courageux - nous illustrons des méthodes plus avancées à l'aide d'un exemple dans lequel nous estimons l'incidence des densités de population les hospitalisations Covid19.

Attention, gardez à l'esprit que cet exemple est uniquement dans le but de s'exercer avec les méthodes de visualisation Python. Nous ne prétendons pas en faire ici une étude scientifique réelle!

#### 3.1 Télécharger les données de superficie et statistiques de population par canton

Source: office fédéral de la statistique et wikipédia

Format: TSV (tab separated)
    
Méthode: [pandas.read_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_table.html), ou pandas.read_csv avec l'option `sep='\t'`.
* Nous utilisons le code d'abréviation du canton comme index.


In [ ]:
cf = pd.read_table("../data/cantons.csv",index_col='Canton')

-----
#### 3.2 Comprendre la structure de la donnée de superficie et statistiques de population

* La table est indexée par canton.
* Les colonnes 3 à 21 indiquent la répartition des types de surface (Ha) du territoire cantonal.
* Les colonnes 22 à 24 sont réspectivement le nombre de résidents, nombre de municipalités, et langue majoritaire du canton.

In [ ]:
cf.info()

In [ ]:
cf.head(5)

-----
#### 3.3 Calcul de densité de population

Quelle est la densité d'habitants par surface **habitable** (colonne _Batiment_)?

Méthode:
* Nous ajoutons une nouvelle colonne à la DataFrame qui est le ratio des colonnes $\frac{Population}{Batiment}$

In [ ]:
cf['Density']=cf.Population/cf.Batiment
cf.head(5)

Barplot de densité de population par canton par ordre décroissant, et histogramme des densités.

In [ ]:
cf.Density.sort_values(ascending=False).plot.bar()

cf.Density.plot.hist()

-----
#### 3.4 Préparation de la donnée Covid19

Nous nous intéressons uniquement au nombre d'hospitalisations pendant le mois d'Octobre 2020. Cette période corréspond à la deuxième vague Covid19, avant que de nouvelles mesures de distances sociales ne soient imposées.

Nous faisons l'hypothèse que le nombre d'hospitalisations dues au Covid19 suit une loi exponentielle $h(t)=e^{\alpha t}$, et nous utlisons la propriété:

$\large \frac{\delta e^{\alpha t}}{\delta t}=\alpha e^{\alpha t}$

pour estimer de façon très simple un facteur de croissance par canton:

$\large \alpha_{canton}(t) \approx \frac{\frac{\Delta h(t)}{\Delta t}}{h(t)}$


Méthode - étape 1: mise en forme des donnée d'hospitalisations
* `current_hosp.dropna(axis=1,thresh=200)`: considère seulement le nombre d'hosptalisations, et ignore tous les cantons (axis=1) pour lesquels nous avons moins de 200 mesures.
* `ewm(halflife='2D',times=dfp.index).mean()`: lissage exponentiel $y_t=\lambda x_t+(1-\lambda) y_{t-1}$, avec $\lambda$ calculé en terme de demie-vie au cours de laquelle une observation $x_t$ décroit de la moitié de sa valeur sur l'axe des temps spécifié par `times=dfp.index` (l'index est la colonne _date_ de la table _dfp_).
* `loc['2020-10-10':'2020-11-05']`: fenêtre de temps de la deuxième vague

In [ ]:
dfp_avg=dfp.current_hosp.dropna(axis=1,thresh=200).ewm(halflife='1D',times=dfp.index).mean().loc['2020-10-10':'2020-11-05']

Nous affichons _dfp\_avg_ sur une échelle logarithmique pour vérifier que nombre d'hospitalisations suit une croissance exponentielle

In [ ]:
dfp_avg.plot(logy=True)

Méthode - étape 2: Calcul de l'approximation $\Delta h(t)/\Delta t/h(t)$, avec $\Delta t=1 $jour.
* `apply(estimate_alpha)`: applique la fonction sur l'ensemble de données de chaque canton.

In [ ]:
def estimate_alpha(x):
    return np.diff(x)/x[1:]

t_alpha=dfp_avg.apply(estimate_alpha)

Nous calculons les _boites à moustaches_ pour visualiser les variations du taux de croissance $\alpha_{canton}(t)$ de Covid19 estimé pour chaque canton.
Nous utilisons ensuite la valeur médiane de $\alpha_{canton}(t)$ sur l'interval de temps considéré pour en déduire $\alpha_{canton}$ si les variations sont raisonnables.

In [ ]:
t_alpha.boxplot()

In [ ]:
alpha=t_alpha.median().to_frame('alpha')
alpha.head(5)

-----
#### 3.5 Jointure de la donnée Covid19 et de densité de population

Nous pouvons maintenant comparer les $\alpha_{canton}$ avec les données de densité d'habitants par zones habitables. Il suffit de faire une _jointure_ pour associer les $\alpha_{canton}$ avec les densités correspondantes.

Notez que la préparation de la donnée constitue 80% à 90% de l'effort de data science. Afin de garder cet exemple simple, nous avons omis des étapes importantes de cette préparation, dont la normalisation des données. Il ne faut donc pas tirer des conclusions hatives à partir de ces résultats.

Méthode 1: [pandas.DataFrame.join](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html)
* `on='canton'`: utilise la colonne _canton_ pour la clé de jointure des données de Covid19
* `how='inner'`

Voir aussi [pandas.merge](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

In [ ]:
pdf=alpha.join(cf.Density,on='canton',how='inner')

In [ ]:
pdf.plot(x='alpha',y='Density',style='^')